In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [ ]:
import os
import sys
import cv2
import json
import glob
import torch
import py_wsi
import tifffile
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from tqdm.notebook import tqdm
from collections import Counter
from matplotlib import pyplot as plt

sys.path.append("../code/")

In [ ]:
from params import *
from utils.rle import *

from data.dataset import load_image

from utils.metrics import dice_scores_img
from utils.plots import plot_heatmap_preds, plot_contours_preds

In [ ]:
import rasterio
IDENTITY = rasterio.Affine(1, 0, 0, 0, 1, 0)

### Load

In [ ]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")

ANNOT_PATH = DATA_PATH + "annotation_v3/"

In [ ]:
PLOT = False
ADD_FC = False
ONLY_FC = True

### Train data

In [ ]:
# new_df = df_mask.copy().set_index('id')

# for id_ in tqdm(df_mask['id']):
#     print(f' -> {id_}')
#     if id_ + ".json" in os.listdir(ANNOT_PATH):        
#         annot = json.load(open(ANNOT_PATH + id_ + ".json", 'r'))
        
#         w, h = df_info[df_info['image_file'] == id_ + '.tiff'][['width_pixels', 'height_pixels']].values[0]
        
#         rle = df_mask[df_mask['id'] == id_]['encoding']
        
# #         mask = enc2mask(rle, (w, h)).astype(np.uint8)  # smh not working
#         mask = np.zeros((h, w), dtype=np.uint8)
        
#         if not ONLY_FC:
#             mask += enc2mask(rle, (w, h)).astype(np.uint8)
        
#         added = 0
#         for info in annot:
#             label = info['properties']['classification']['name']

#             if label == "FC" and not ONLY_FC:
#                 if not ADD_FC or id_ != "aaa6a05cc":
#                     continue
                    
#             if ONLY_FC and label != "FC":
#                 continue

#             poly = info['geometry']['coordinates']
#             try:
#                 mask = cv2.fillPoly(mask, np.int32([poly]), True)
#             except ValueError:
#                 poly = np.concatenate([np.array(poly[i]).squeeze() for i in range(len(poly))])
#                 mask = cv2.fillPoly(mask, np.int32([poly]), True)
#             added +=1
            
#         print(f"Added {added} glomerulis")
        
#         new_df.loc[id_] = rle_encode_less_memory(mask)
        
#         if PLOT:
#             img = load_image(os.path.join(TIFF_PATH_4, id_ + ".tiff"), full_size=False)
            
#             mask = cv2.resize(
#                 mask,
#                 (w // 4, h // 4),
#                 interpolation=cv2.INTER_NEAREST,
#             )
#             assert mask.shape == img.shape[:2], (mask.shape, img.shape)
        
#             fig = plot_contours_preds(img, mask, w=1, downsize=4)
#             w = 1000
#             h = int(w *  mask.shape[0] / mask.shape[1])
#             fig.update_layout(
#                 autosize=False,
#                 width=w,
#                 height=h,
#             )

#             fig.show()

#             break

# if not PLOT:
#     name = "train_fix.csv" if not ADD_FC else "train_fc.csv"
#     if ONLY_FC:
#         name = "train_onlyfc.csv"
#     new_df.to_csv(DATA_PATH + name)
    
#     print(f'\n -> Saved masks to {DATA_PATH + name}')

### Extra data

In [ ]:
PLOT = False
SAVE_TIFF = False
SAVE = True
ADD_FC = False
ONLY_FC = False

In [ ]:
files = [p for p in os.listdir(DATA_PATH + "extra/") if p.endswith("svs")]
rles = {}

for file in tqdm(files):
    id_ = file[:-4]
    print(f' -> {id_}')
    
#     if id_ != "SAS_21908_001":
#         continue
    
    if os.path.exists(ANNOT_PATH + id_ + ".json"):
        original_img = rasterio.open(DATA_PATH + "extra/" + file, transform=IDENTITY, num_threads='all_cpus')
        img = original_img.read([1, 2, 3]).transpose(1, 2, 0).astype(np.uint8)

        shape = img.shape[:2]
        
        annot = json.load(open(ANNOT_PATH + id_ + ".json", 'r'))

        mask = np.zeros(shape, dtype=np.uint8)

        added = 0
        for info in annot:
            poly = np.array(info['geometry']['coordinates'])
            
            try:
                label = info['properties']['classification']['name']
            except KeyError:
                print('??')
                label = "G"
            
            if not ADD_FC and label == "FC":
                continue

            if ONLY_FC and label != "FC":
                continue
                
            poly = info['geometry']['coordinates']
            try:
                mask = cv2.fillPoly(mask, np.int32([poly]), True)
            except ValueError:
                poly = np.concatenate([np.array(poly[i]).squeeze() for i in range(len(poly))])
                mask = cv2.fillPoly(mask, np.int32([poly]), True)
            added += 1
        
        print(f"Added {added} glomerulis")
        
        if PLOT:
            print('plot')
            fig = plot_contours_preds(img, mask, w=2, downsize=8)

            w = 1000
            h = int(w *  mask.shape[0] / mask.shape[1])
            fig.update_layout(
                autosize=False,
                width=w,
                height=h,
            )

            fig.show()

            break
            
        if SAVE:
            if SAVE_TIFF:
                img = cv2.resize(
                    img,
                    (img.shape[1] // 2, img.shape[0] // 2),
                    interpolation=cv2.INTER_AREA,
                )
                    
                if not os.path.exists(DATA_PATH + "extra_tiff/"):
                    os.mkdir(DATA_PATH + "extra_tiff/")
                tifffile.imsave(DATA_PATH + "extra_tiff/" + f"{id_}.tiff", img)

            mask = cv2.resize(
                mask,
                (mask.shape[1] // 2, mask.shape[0] // 2),
                interpolation=cv2.INTER_NEAREST,
            )

            rles[id_] = rle_encode_less_memory(mask)

In [ ]:
df_annot_extra = pd.DataFrame.from_dict(rles, orient='index', columns=['encoding'])

if SAVE and not PLOT:
    name = "train_extra.csv" if not ADD_FC else "train_extra_fc.csv"
    if ONLY_FC:
        name = "train_extra_onlyfc.csv"
    df_annot_extra.to_csv(DATA_PATH + name)
    print(f'\n -> Saved masks to {DATA_PATH + name}')